### 2 Bank Summary
<br>


In [1]:
import pandas as pd
pd.options.display.float_format = '{:,.2f}'.format

%store -r bank

<br>




### Balances

In [2]:
beg_bal = bank[bank['Desc'].str.startswith('Beginning')].set_index('Month', drop=True)
beg_bal = beg_bal['Balance']

In [3]:
end_bal = bank[bank['Desc'].str.startswith('Ending')].set_index('Month', drop=True)
end_bal = end_bal['Balance']

<br>

### LOC

In [4]:
loan_p = bank[bank['Desc'].str.contains('PAYMENT TO COMMERCIAL LOAN')]
loan_pay = loan_p.groupby('Month')['Amount'].agg('sum')

In [5]:
loan_a = bank[bank['Desc'].str.contains('ADVANCE FROM COMMERCIAL LOAN')]
loan_adv = loan_a.groupby('Month')['Amount'].agg('sum')

<br>

### Payroll Transfers

In [6]:
plugs = {'Aug': -462362.76, 'Sep': -426070.38, 'Oct': -359172.16,
         'Nov': -458700, 'Dec': -355600}

#<----index of line items supporting above totals
plug_indices = [14,24,25,55,56,101,102,136,146,224,244,256,274,
                295,306,327,337,369,391,402,445,474,505,522,557,583]

PR_x = pd.Series(plugs)

<br>

### Checks / Other Payments

In [7]:
chex = bank[(bank['Desc'].str.startswith('CHECK'))&(bank['Amount'] <  0)]
checks = chex.groupby('Month')['Amount'].agg('sum')


 - Other Payments does not include checks, commercial loan payments, and PR transfers (<b>per the schedule provided</b>).

In [8]:
OP = bank[~(bank['Desc'].str.startswith('CHECK'))&(bank['Amount'] <  0)]
OP_adj = OP[~(OP.index.isin(plug_indices))]

other_pay = OP_adj[~(OP_adj['Desc'].str.contains('PAYMENT TO COMMERCIAL LOAN'))]
other_pay = other_pay.groupby('Month')['Amount'].agg('sum')

<br>

### Deposits

 - does not include commercial loan advances

In [9]:
credits = bank[~(bank['Desc'].str.contains('ADVANCE FROM COMMERCIAL LOAN'))&(bank['Amount'] >  0)]
deposits = credits.groupby('Month')['Amount'].agg('sum')

<br>

### Bank Summary

In [10]:
b_summary = pd.concat([beg_bal, deposits, checks, other_pay, loan_adv, 
                     loan_pay, PR_x, end_bal],axis=1)
b_summary.columns = ['Beg','Deposits','Checks','Other_Pay','CL_Adv','CL_Pay','PR_x','End']

b_summary = b_summary.T
b_summary

,Aug,Sep,Oct,Nov,Dec
Beg,950.16,"-120,862.21",973.04,583.78,"132,910.71"
Deposits,"2,309,855.08","1,457,343.03","1,173,557.77","2,980,452.96","2,384,075.32"
Checks,"-1,359,231.21","-229,753.13","-214,589.03","-848,381.37","-918,788.15"
Other_Pay,"-976,694.08","-454,060.61","-604,350.70","-980,855.98","-1,370,408.81"
CL_Adv,"1,645,500.00","1,269,000.00","884,500.00","899,500.00","1,020,000.00"
CL_Pay,"-1,278,879.40","-1,494,623.66","-880,335.14","-1,459,688.68","-891,870.95"
PR_x,"-462,362.76","-426,070.38","-359,172.16","-458,700.00","-355,600.00"
End,"-120,862.21",973.04,583.78,"132,910.71",318.12


<br>
<br>



#### create Amount Frequency structure
 - added for use in Part 4

In [11]:
credits = bank[~(bank['Desc'].str.contains('ADVANCE FROM COMMERCIAL LOAN'))&(bank['Amount'] >  0)]

chex = bank[(bank['Desc'].str.startswith('CHECK'))&(bank['Amount'] <  0)]

OP = bank[~(bank['Desc'].str.startswith('CHECK'))&(bank['Amount'] <  0)]
OP_adj = OP[~(OP.index.isin(plug_indices))]
other_pay = OP_adj[~(OP_adj['Desc'].str.contains('PAYMENT TO COMMERCIAL LOAN'))]

loan_a = bank[bank['Desc'].str.contains('ADVANCE FROM COMMERCIAL LOAN')]

loan_p = bank[bank['Desc'].str.contains('PAYMENT TO COMMERCIAL LOAN')]

PR_Transfers = bank[bank.index.isin(plug_indices)]


cats = {'Credits': credits, 'Checks': chex, 'Other_Pay': other_pay,
       'CL_Adv': loan_a, 'CL_Pay': loan_p, 'PR_x': PR_Transfers}

bank_pool = {}

for k,v in cats.items():
    mo_hist = []
    for month in list(bank['Month'].unique()):
        results = v[v['Month'] == month].copy() 
        mo_hist.append(results) 
        
    bank_pool[k] = mo_hist       

In [12]:
%store b_summary
%store bank_pool

Stored 'b_summary' (DataFrame)
Stored 'bank_pool' (dict)
